## Data Processing Peak Season

In [137]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players
from college_info_puller import CollegeInfo
import pandas as pd
import numpy as np
import pickle
import time
import requests

In [118]:
nba_03 = pd.read_csv('Data/season_dat/NBA_2003-04.csv')
nba_04 = pd.read_csv('Data/season_dat/NBA_2004-05.csv')
nba_05 = pd.read_csv('Data/season_dat/NBA_2005-06.csv')
nba_06 = pd.read_csv('Data/season_dat/NBA_2006-07.csv')
nba_07 = pd.read_csv('Data/season_dat/NBA_2007-08.csv')
nba_08 = pd.read_csv('Data/season_dat/NBA_2008-09.csv')
nba_09 = pd.read_csv('Data/season_dat/NBA_2009-10.csv')
nba_10 = pd.read_csv('Data/season_dat/NBA_2010-11.csv')
nba_11 = pd.read_csv('Data/season_dat/NBA_2011-12.csv')
nba_12 = pd.read_csv('Data/season_dat/NBA_2012-13.csv')
nba_13 = pd.read_csv('Data/season_dat/NBA_2013-14.csv')
nba_14 = pd.read_csv('Data/season_dat/NBA_2014-15.csv')
nba_15 = pd.read_csv('Data/season_dat/NBA_2015-16.csv')
nba_16 = pd.read_csv('Data/season_dat/NBA_2016-17.csv')
nba_17 = pd.read_csv('Data/season_dat/NBA_2017-18.csv')
nba_18 = pd.read_csv('Data/season_dat/NBA_2018-19.csv')
nba_19 = pd.read_csv('Data/season_dat/NBA_2019-20.csv')
nba_20 = pd.read_csv('Data/season_dat/NBA_2020-21.csv')
nba_21 = pd.read_csv('Data/season_dat/NBA_2021-22.csv')
nba_22 = pd.read_csv('Data/season_dat/NBA_2022-23.csv')
nba_23 = pd.read_csv('Data/season_dat/NBA_2023-24.csv')
nba_24 = pd.read_csv('Data/season_dat/NBA_2024-25.csv')

nba = pd.concat([nba_03,nba_04,nba_05,nba_06,nba_07,nba_08,nba_09,nba_10,
                 nba_11,nba_12,nba_13,nba_14,nba_15,nba_16,nba_17,nba_18,nba_19,nba_20,
                 nba_21,nba_22,nba_23,nba_24], ignore_index=True, sort=False)

In [63]:
num_seasons = nba['Player'].value_counts()

Let's get career data for those who have played in the NBA recently. Say over the last 10 years who were the players who got some playing time? The cut off will be set at 250 min a season or 3 min per game to allow for some fringe players or those who got injured to be included in our data set. 

Then we'll pull the players peak years as a measure of their true ability

In [142]:
nba_recent = pd.concat([nba_24,nba_23,nba_22,nba_21,nba_20,nba_19,nba_18,nba_17,nba_16,nba_15], ignore_index=True, sort=False)
rec_fil = nba_recent[nba_recent['MP']>=250]
cur_players = rec_fil.groupby('Player')['MP'].sum().sort_values().index
all_players = nba.groupby('Player')['MP'].sum().index

In [ ]:
nba_bpms= nba.groupby('Player')['BPM'].unique()
nba_ages = nba.groupby('Player')['Age'].unique()

#best season by bpm
bpm_maxes = []

#will take average of up to a players 3 best bpm seasons, avg of 2 or 1 if they haven't been in league that long
avg_best_bpms = []

#starting age when debuting
age_enter = []

#age when peaked
age_peaked = []

for p in cur_players:
    if p in all_players:
        bpms = nba_bpms.loc[p]
        peaked = max(bpms)
        peaked_loc = bpms.argmax()
        
        bpm_maxes.append(peaked)
        
        #up to
        top3_bpms = np.sort(bpms)[::-1][:3]
        
        avg_best_bpms.append(np.mean(top3_bpms))
        
        ages = nba_ages.loc[p]
        age_enter.append(min(ages))

        age_peaked.append(ages[peaked_loc])
        
        
peaked_df = pd.DataFrame({'Player': cur_players,
              'Age Enter': age_enter,
              'Max BPM': bpm_maxes,
              'Age Peaked': age_peaked,
              'Top 3 BPM Avg': avg_best_bpms})
        

In [140]:
info_dict = CollegeInfo().info_dict

player_w_info = list(info_dict.keys())

missing_info = []

for player in peaked_df['Player']:
    if player not in player_w_info:
        missing_info.append(player)

In [146]:
collegefetcher = CollegeInfo()

updated_info = collegefetcher.fill_in_info(
    cur_players=missing_info,
    player_w_info=player_w_info
)

Could not find: R.J. Hunter
Could not find: J.J. Hickson
Could not find: K.J. McDaniels
Could not find: P.J. Hairston
Could not find: J.R. Smith
Saving pulled info...


In [153]:
retry_list = ['RJ Hunter', 'JJ Hickson', 'KJ McDaniels', 'PJ Hairston', 'JR Smith']

updated_info = collegefetcher.fill_in_info(
    cur_players=retry_list,
    player_w_info=player_w_info
)

Saving pulled info...


In [ ]:
info_dict = collegefetcher.info_dict

del info_dict['J.R. Smith']
del info_dict['R.J. Hunter']
del info_dict['J.J. Hickson']
del info_dict['K.J. McDaniels']
del info_dict['P.J. Hairston']

with open(collegefetcher.info_dict_path, 'wb') as f:
    pickle.dump(info_dict, f)
    


In [192]:
updated_info = collegefetcher.fill_in_info(
    cur_players=['R.J. Hampton'],
    player_w_info=player_w_info
)

Saving pulled info...


In [ ]:
players.find_players_by_full_name('R.J. Hampton')

player_info = collegefetcher.get_player_info(1630181)

info = player_info.common_player_info.get_data_frame()

college = info.loc[0, "SCHOOL"]
draft_pick = info.loc[0, "DRAFT_NUMBER"]

info_dict['R.J. Hampton'] = {
    'College': college,
    'Draft Pick': draft_pick
}

In [199]:
players.find_players_by_full_name(player)

player_info = collegefetcher.get_player_info(201942)

info = player_info.common_player_info.get_data_frame()

college = info.loc[0, "SCHOOL"]
draft_pick = info.loc[0, "DRAFT_NUMBER"]

info_dict[player] = {
    'College': college,
    'Draft Pick': draft_pick
}

In [200]:
info_dict['DeMar DeRozan']

{'College': 'Southern California', 'Draft Pick': '9'}

In [201]:
with open(collegefetcher.info_dict_path, 'wb') as f:
    pickle.dump(info_dict, f)

## Adding Draft Features

In [ ]:
rename = {'R.J. Hunter': 'RJ Hunter',
          'J.J. Hickson': 'JJ Hickson',
          'K.J. McDaniels': 'KJ McDaniels',
          'P.J. Hairston': 'PJ Hairston',
          'J.R. Smith': 'JR Smith'}

peaked_df['Player'] = peaked_df['Player'].replace(rename)

In [ ]:
peaked_df['College'] = peaked_df['Player'].map(lambda x: info_dict[x]['College'])
peaked_df['Draft Pick'] = peaked_df['Player'].map(lambda x: info_dict[x]['Draft Pick'])

In [234]:
peaked_df.loc[0,'Draft Pick']
peaked_df.loc[peaked_df['Draft Pick'].isnull(),'Draft Pick'] = 0
peaked_df['Draft Pick'] = peaked_df['Draft Pick'].replace({'Undrafted':0})
peaked_df['Draft Pick'] = peaked_df['Draft Pick'].astype(int)

In [238]:
rounds = [] 
round_bins = []

for i in range(len(peaked_df)):
    if peaked_df.loc[i,'Draft Pick'] > 0 and peaked_df.loc[i,'Draft Pick'] <= 30:
        rounds.append('1st')
        if peaked_df.loc[i,'Draft Pick'] <= 14:
            round_bins.append('1-14')
        else:
            round_bins.append('15-30')
    elif peaked_df.loc[i,'Draft Pick'] > 30 and peaked_df.loc[i,'Draft Pick'] <= 60:
        rounds.append('2nd')
        if peaked_df.loc[i,'Draft Pick'] <= 44:
            round_bins.append('31-44')
        else:
            round_bins.append('45-60')
    else:
        rounds.append('Undrafted')
        round_bins.append('Undrafted')

peaked_df['Round'] = rounds
peaked_df['Round Bin'] = round_bins
peaked_df.head()

,Player,Age Enter,Max BPM,Age Peaked,Top 3 BPM Avg,College,Draft Pick,Round,Round Bin
0,Pat Spencer,27,-1.6,28,-4.0,Northwestern,0,Undrafted,Undrafted
1,Jeremy Evans,23,5.0,24,3.8,Western Kentucky,55,2nd,45-60
2,Cam Spencer,24,0.5,24,0.5,Connecticut,53,2nd,45-60
3,Marcus Bagley,23,-4.3,23,-4.3,Arizona State,0,Undrafted,Undrafted
4,Julian Washburn,27,-4.6,27,-4.6,Texas-El Paso,0,Undrafted,Undrafted


In [245]:
peaked_df.to_csv('Data/peaks.csv')